## Load 311 Data Locally

In [1]:
import pyspark as ps
from pyspark.sql.types import *
from pyspark.sql import functions as SparkFunc
from pyspark.mllib.tree import RandomForest, RandomForestModel ,GradientBoostedTrees, GradientBoostedTreesModel
from pyspark.mllib.util import MLUtils
from pyspark.sql.functions import unix_timestamp

In [68]:
spark = ps.sql.SparkSession.builder \
        .appName("df_311") \
        .getOrCreate()


In [69]:
sc = spark.sparkContext

In [70]:
df = spark.read.format("csv").option("header", "true").load("../data/311_Service_Requests_from_2010_to_Present.csv")

In [71]:
df.printSchema()

root
 |-- Unique Key: string (nullable = true)
 |-- Created Date: string (nullable = true)
 |-- Closed Date: string (nullable = true)
 |-- Agency: string (nullable = true)
 |-- Agency Name: string (nullable = true)
 |-- Complaint Type: string (nullable = true)
 |-- Descriptor: string (nullable = true)
 |-- Location Type: string (nullable = true)
 |-- Incident Zip: string (nullable = true)
 |-- Incident Address: string (nullable = true)
 |-- Street Name: string (nullable = true)
 |-- Cross Street 1: string (nullable = true)
 |-- Cross Street 2: string (nullable = true)
 |-- Intersection Street 1: string (nullable = true)
 |-- Intersection Street 2: string (nullable = true)
 |-- Address Type: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Landmark: string (nullable = true)
 |-- Facility Type: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Due Date: string (nullable = true)
 |-- Resolution Description: string (nullable = true)
 |-- Resolution Action

In [72]:
#Select only needed cols as found in Pandas EDA
df_rdd = df.select('Created Date','Agency','Closed Date','Complaint Type',\
                   'Descriptor','Borough','Community Board','Open Data Channel Type','Status','Latitude','Longitude')


In [73]:
df_rdd.printSchema()

root
 |-- Created Date: string (nullable = true)
 |-- Agency: string (nullable = true)
 |-- Closed Date: string (nullable = true)
 |-- Complaint Type: string (nullable = true)
 |-- Descriptor: string (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Community Board: string (nullable = true)
 |-- Open Data Channel Type: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)



In [74]:
# Function takes col name as string, recasts it to a temporary col as the specified data type string, then drops origional col and renames the 
# tempoary one
def convert_rdd_type(orig_rdd,column_name_str,data_type_str):
    return (orig_rdd.withColumn("{}_temp".format(column_name_str), 
                                df_rdd[column_name_str].cast(data_type_str)).drop(
                                column_name_str)).withColumnRenamed("{}_temp".format(column_name_str),
                                 column_name_str)

In [52]:
# #Convert to proper data types
# # df_rdd= df_rdd.withColumn("Latitude_temp", df_rdd.Latitude.cast('Float')).drop("Latitude").withColumnRenamed("Latitude_temp", "Latitude")
# df_rdd = df_rdd.withColumn("Longitude_temp", df_rdd['Longitude'].cast('Float'))

In [75]:
df_rdd = convert_rdd_type(df_rdd,'Longitude','Float')
df_rdd = convert_rdd_type(df_rdd,'Latitude','Float')
# df_rdd = convert_rdd_type(df_rdd,'Created Date','Date')

In [110]:
#convert to unix timestamp for Created & Closed
timeFmt = "MM-dd-yyyy'T'HH:mm:ss.SSS"
time_test = SparkFunc.unix_timestamp(df_rdd['Closed Date'], format=timeFmt)
df_rdd = df_rdd.withColumn('test',time_test)
# timeDiff = (F.unix_timestamp('EndDateTime', format=timeFmt)
#             - F.unix_timestamp('StartDateTime', format=timeFmt))
# df = df.withColumn("Duration", timeDiff)


In [76]:
df_rdd.printSchema()

root
 |-- Created Date: string (nullable = true)
 |-- Agency: string (nullable = true)
 |-- Closed Date: string (nullable = true)
 |-- Complaint Type: string (nullable = true)
 |-- Descriptor: string (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Community Board: string (nullable = true)
 |-- Open Data Channel Type: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Longitude: float (nullable = true)
 |-- Latitude: float (nullable = true)



In [87]:
from pyspark.sql.functions import col
from pyspark.sql.functions import to_timestamp, date_format
spark.conf.set('spark.sql.session.timeZone', 'UTC')

In [104]:
df_rdd.select(
        to_timestamp(df_rdd['Created Date'], "MM/dd/yyyy HH:mm:ss"), #SUPER SUPER SPECIFIC FORMATING
    ).alias('timestamp_value').show()

+---------------------------------------------------+
|to_timestamp(`Created Date`, 'MM/dd/yyyy HH:mm:ss')|
+---------------------------------------------------+
|                                2015-08-10 09:18:31|
|                                2015-08-10 12:20:43|
|                                2015-08-10 06:16:29|
|                                2015-08-10 10:50:00|
|                                2015-08-10 10:36:00|
|                                2015-08-10 10:12:00|
|                                2015-08-10 01:17:30|
|                                2015-08-10 06:41:33|
|                                2015-08-10 08:10:06|
|                                2015-08-10 01:17:46|
|                                2015-08-10 10:55:00|
|                                2015-08-10 04:12:00|
|                                2015-08-10 12:37:03|
|                                2015-08-10 06:51:18|
|                                2015-08-10 02:40:53|
|                           

In [84]:
df_rdd.take(1)

[Row(Created Date='08/10/2015 09:18:31 AM', Agency='HPD', Closed Date='08/13/2015 12:46:56 PM', Complaint Type='PLUMBING', Descriptor='STEAM PIPE/RISER', Borough='BRONX', Community Board='04 BRONX', Open Data Channel Type='PHONE', Status='Closed', Longitude=-73.91136169433594, Latitude=40.84086227416992)]

## Dask

In [2]:
import pandas as pd
import dask.dataframe as dd
from multiprocessing.pool import ThreadPool
import os
from sodapy import Socrata
import json

In [3]:
sodapy_token = 'tvPeTjPatFwjuelfpMNb0G8WH'
sodapy_domain = 'data.cityofnewyork.us'
database_311 = "fhrw-4uyv"
query = "created_date > '2018-01-01T00:00:00.000'"

In [4]:
select_sql = "agency,borough,closed_date,community_board,complaint_type,created_date,descriptor,open_data_channel_type,status,longitude,latitude" 

In [5]:
client = Socrata(sodapy_domain, sodapy_token)

In [6]:
results = client.get(database_311, select=select_sql, where=query, limit=500000)

In [7]:
df = pd.DataFrame.from_records(results)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 11 columns):
agency                    500000 non-null object
borough                   500000 non-null object
closed_date               487874 non-null object
community_board           500000 non-null object
complaint_type            500000 non-null object
created_date              500000 non-null object
descriptor                498163 non-null object
latitude                  490390 non-null object
longitude                 490390 non-null object
open_data_channel_type    500000 non-null object
status                    500000 non-null object
dtypes: object(11)
memory usage: 42.0+ MB


In [9]:
#drop nans:
df.dropna(subset=['closed_date'],inplace=True)

In [10]:
df.dropna(subset=['descriptor'],inplace=True)

In [11]:
df.dropna(subset=['latitude','longitude'],inplace=True)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 477139 entries, 8 to 499999
Data columns (total 11 columns):
agency                    477139 non-null object
borough                   477139 non-null object
closed_date               477139 non-null object
community_board           477139 non-null object
complaint_type            477139 non-null object
created_date              477139 non-null object
descriptor                477139 non-null object
latitude                  477139 non-null object
longitude                 477139 non-null object
open_data_channel_type    477139 non-null object
status                    477139 non-null object
dtypes: object(11)
memory usage: 43.7+ MB


In [13]:
community_board_list = set(['06 BRONX', '01 BRONX', '14 QUEENS', '13 QUEENS',
        '13 BROOKLYN', '09 BROOKLYN', '10 QUEENS',
       '08 BRONX', '10 BRONX', '01 QUEENS', '11 QUEENS', '01 BROOKLYN',
       '12 BRONX', '14 BROOKLYN', '07 MANHATTAN', '04 MANHATTAN',
       '05 MANHATTAN', '07 BRONX', '06 QUEENS', '18 BROOKLYN',
       '02 STATEN ISLAND', '15 BROOKLYN', '07 QUEENS', '03 STATEN ISLAND',
       '03 QUEENS', '08 QUEENS', '01 STATEN ISLAND', '07 BROOKLYN',
       '04 BRONX', '12 BROOKLYN', '12 QUEENS',
       '17 BROOKLYN', '04 QUEENS', '03 BROOKLYN', '05 QUEENS',
       '02 QUEENS', '03 MANHATTAN', '11 BRONX', '10 BROOKLYN',
       '06 BROOKLYN', '11 BROOKLYN', '05 BROOKLYN', '04 BROOKLYN',
       '12 MANHATTAN', '09 QUEENS', '09 MANHATTAN', '10 MANHATTAN',
       '16 BROOKLYN', '09 BRONX', '05 BRONX', '02 BROOKLYN',
       '06 MANHATTAN', '02 BRONX', '08 BROOKLYN', '11 MANHATTAN',
       '08 MANHATTAN', '02 MANHATTAN',  '03 BRONX',
       '01 MANHATTAN'])

In [14]:
df = df[df['community_board'].isin(community_board_list)]

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 472385 entries, 8 to 499999
Data columns (total 11 columns):
agency                    472385 non-null object
borough                   472385 non-null object
closed_date               472385 non-null object
community_board           472385 non-null object
complaint_type            472385 non-null object
created_date              472385 non-null object
descriptor                472385 non-null object
latitude                  472385 non-null object
longitude                 472385 non-null object
open_data_channel_type    472385 non-null object
status                    472385 non-null object
dtypes: object(11)
memory usage: 43.2+ MB


In [16]:
df = df[df['status'] == 'Closed']

In [120]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 463513 entries, 8 to 499999
Data columns (total 15 columns):
agency                      463513 non-null object
borough                     463513 non-null object
community_board             463513 non-null object
complaint_type              463513 non-null object
descriptor                  463513 non-null object
latitude                    463513 non-null object
longitude                   463513 non-null object
open_data_channel_type      463513 non-null object
status                      463513 non-null object
time_till_resolution        463513 non-null float64
created_date_year           463513 non-null int64
created_date_month          463513 non-null int64
created_date_day            463513 non-null int64
created_date_hour           463513 non-null int64
created_date_day_of_week    463513 non-null int64
dtypes: float64(1), int64(5), object(9)
memory usage: 56.6+ MB


In [18]:
created = pd.to_datetime(df['created_date'],utc=True)
closed = pd.to_datetime(df['closed_date'],utc=True)
df['time_till_resolution'] =created - closed
df['time_till_resolution'] =(-round(
    df['time_till_resolution'].astype('timedelta64[s]')/3600,
        2))

In [19]:
df.drop('closed_date', axis=1 , inplace=True)

In [20]:
df['created_date']= pd.to_datetime(df['created_date'])

In [21]:
df['created_date_year'] = df['created_date'].dt.year
df['created_date_month'] = df['created_date'].dt.month
df['created_date_day'] = df['created_date'].dt.day
df['created_date_hour'] = df['created_date'].dt.hour
df['created_date_day_of_week'] = df['created_date'].dt.dayofweek

In [22]:
df.head(10)

,agency,borough,community_board,complaint_type,created_date,descriptor,latitude,longitude,open_data_channel_type,status,time_till_resolution,created_date_year,created_date_month,created_date_day,created_date_hour,created_date_day_of_week
8,TLC,MANHATTAN,03 MANHATTAN,Taxi Complaint,2018-04-22 14:14:21,Insurance Information Requested,40.722933781818995,-73.98863937450832,PHONE,Closed,121.67,2018,4,22,14,6
9,DSNY,BROOKLYN,11 BROOKLYN,Graffiti,2018-01-12 16:53:27,Graffiti,40.61039710713843,-73.98224405571739,UNKNOWN,Closed,2119.11,2018,1,12,16,4
10,DSNY,BROOKLYN,07 BROOKLYN,Graffiti,2018-01-12 11:43:22,Graffiti,40.64520431702971,-74.01388064443375,UNKNOWN,Closed,2076.28,2018,1,12,11,4
11,DSNY,BROOKLYN,11 BROOKLYN,Graffiti,2018-01-13 15:38:09,Graffiti,40.6121174675791,-73.97857346145312,UNKNOWN,Closed,2048.36,2018,1,13,15,5
12,DSNY,QUEENS,03 QUEENS,Graffiti,2018-01-17 11:34:38,Graffiti,40.75773878300173,-73.89143468976863,UNKNOWN,Closed,1812.42,2018,1,17,11,2
13,DSNY,QUEENS,05 QUEENS,Graffiti,2018-01-18 06:05:24,Graffiti,40.72139592005145,-73.90428648490621,UNKNOWN,Closed,1985.91,2018,1,18,6,3
24,DSNY,QUEENS,04 QUEENS,Request Large Bulky Item Collection,2018-04-23 14:57:00,Request Large Bulky Item Collection,40.749481408215175,-73.85760529263457,PHONE,Closed,81.05,2018,4,23,14,0
25,DSNY,QUEENS,04 QUEENS,Request Large Bulky Item Collection,2018-04-24 00:11:00,Request Large Bulky Item Collection,40.746846803886804,-73.86013721874471,PHONE,Closed,47.82,2018,4,24,0,1
28,NYPD,BRONX,02 BRONX,Illegal Parking,2018-05-02 13:58:23,Double Parked Blocking Vehicle,40.82755236249353,-73.8958411054741,PHONE,Closed,5.62,2018,5,2,13,2
46,DSNY,BRONX,02 BRONX,Graffiti,2018-01-03 08:02:22,Graffiti,40.80916622515735,-73.88507250486421,UNKNOWN,Closed,2559.96,2018,1,3,8,2


In [23]:
df.open_data_channel_type.value_counts()

PHONE      265084
ONLINE     108926
MOBILE      51524
UNKNOWN     34347
OTHER        3632
Name: open_data_channel_type, dtype: int64

In [24]:
df.agency.value_counts()

HPD      153259
NYPD     135050
DSNY      58787
DEP       40855
DOT       37257
DOB       12400
DOHMH      7311
DPR        7303
TLC        5394
DCA        2923
DHS        1360
DOF         762
DOE         617
EDC          93
DOITT        93
DFTA         49
Name: agency, dtype: int64

In [25]:
df[df.agency=='DCA'].time_till_resolution.mean()

211.5026137529937

In [26]:
df.groupby('community_board')

In [27]:
HPD = 234/24

In [28]:
#Make copy
df_test = df

In [119]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 463513 entries, 8 to 499999
Columns: 958 entries, created_date_year to open_data_channel_type_UNKNOWN
dtypes: int64(5), uint8(953)
memory usage: 442.5 MB


In [30]:
#Drop status & lat/long
# df_test.drop(['status','latitude','longitude','created_date'], axis=1 , inplace=True)
df_test.drop('created_date', axis=1 , inplace=True)

In [117]:
df_test.head(10)

,created_date_year,created_date_month,created_date_day,created_date_hour,created_date_day_of_week,agency_DCA,agency_DEP,agency_DFTA,agency_DHS,agency_DOB,...,descriptor_Wrong Amount Paid or Withdrawn,descriptor_Yield,descriptor_Zoning - Non-Conforming/Illegal Vehicle Storage,descriptor_installation of hydrant side post (WHFP),descriptor_unknown odor/taste in drinking water (QA6),open_data_channel_type_MOBILE,open_data_channel_type_ONLINE,open_data_channel_type_OTHER,open_data_channel_type_PHONE,open_data_channel_type_UNKNOWN
8,2018,4,22,14,6,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
9,2018,1,12,16,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
10,2018,1,12,11,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
11,2018,1,13,15,5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
12,2018,1,17,11,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
13,2018,1,18,6,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
24,2018,4,23,14,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
25,2018,4,24,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
28,2018,5,2,13,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
46,2018,1,3,8,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [32]:
df_test= pd.get_dummies(df_test, columns=['agency','borough','community_board',
                                      'complaint_type','descriptor','open_data_channel_type'])

In [39]:
df_test.drop('status', axis=1, inplace=True)

In [42]:
df_test.drop(['latitude','longitude'], axis=1,inplace=True)

In [43]:
column_names = set(df_test.columns)

In [45]:
df_test.shape

(463513, 959)

In [48]:
joblib.dump(column_names, 'feature_names.pkl')

['feature_names.pkl']

In [49]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 463513 entries, 8 to 499999
Columns: 959 entries, time_till_resolution to open_data_channel_type_UNKNOWN
dtypes: float64(1), int64(5), uint8(953)
memory usage: 446.0 MB


In [135]:
df.to_csv("../data/cleaned_500k")

In [50]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor , GradientBoostingRegressor
from sklearn.model_selection import train_test_split

In [51]:
y = df_test.pop('time_till_resolution')
X = df_test

In [52]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X,y)

In [118]:
y

8          121.67
9         2119.11
10        2076.28
11        2048.36
12        1812.42
13        1985.91
24          81.05
25          47.82
28           5.62
46        2559.96
47        1807.63
48        2005.89
49        1071.99
50        1790.40
51        1623.62
52         702.00
53         177.53
54         137.33
55         125.42
56         174.22
58          32.90
61         130.65
65          27.13
66          14.33
67          28.88
68        2746.78
69        2850.19
70        1891.73
71        2483.30
72        2031.98
           ...   
499970      63.28
499971     445.30
499972     259.65
499973     525.54
499974       2.91
499975      69.65
499976      49.88
499977     230.90
499978     270.05
499979     218.10
499980     596.17
499981     381.97
499982      24.00
499983     173.37
499984       1.87
499985       3.68
499986      46.88
499987      11.13
499988      64.74
499989      49.31
499990     464.53
499991      -0.00
499992     112.28
499993      50.17
499994    

In [53]:
Xtrain.shape

(347634, 958)

In [106]:
random_forest.predict(try_1)

array([173.101,  27.411, 219.341, 981.5  , 176.069])

In [54]:
Xtrain.head()

,created_date_year,created_date_month,created_date_day,created_date_hour,created_date_day_of_week,agency_DCA,agency_DEP,agency_DFTA,agency_DHS,agency_DOB,...,descriptor_Wrong Amount Paid or Withdrawn,descriptor_Yield,descriptor_Zoning - Non-Conforming/Illegal Vehicle Storage,descriptor_installation of hydrant side post (WHFP),descriptor_unknown odor/taste in drinking water (QA6),open_data_channel_type_MOBILE,open_data_channel_type_ONLINE,open_data_channel_type_OTHER,open_data_channel_type_PHONE,open_data_channel_type_UNKNOWN
83514,2018,5,11,14,4,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
295518,2018,1,15,15,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
197127,2018,1,2,12,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
462144,2018,2,12,16,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
164752,2018,4,29,14,6,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [55]:
random_forest = RandomForestRegressor(n_jobs = -1)
gradient_boost = GradientBoostingRegressor()

In [56]:
random_forest.fit(Xtrain,ytrain)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [57]:
random_forest.score(Xtest,ytest)

0.5652564371000576

In [47]:
from sklearn.externals import joblib

In [58]:
joblib.dump(random_forest, 'random_forest_model.pkl')

['random_forest_model.pkl']

In [80]:
gradient_boost = GradientBoostingRegressor()

In [82]:
gradient_boost.fit(Xtrain, ytrain)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

In [87]:
predictions = gradient_boost.predict(Xtest)

In [88]:
predictions[:10]

array([9.97578163e-01, 5.80281648e+00, 7.54037873e+00, 9.97578163e-01,
       1.12450649e+03, 3.21011996e-01, 1.86189449e+02, 6.04520044e+01,
       8.08189636e-01, 6.98806956e+01])

In [86]:
ytest

367230       0.87
122449       6.02
382313       7.63
30525        0.87
208742    1145.95
84224       -0.00
66651      182.31
144042      63.69
27797        0.69
117725      69.61
15829      121.10
253025      39.85
16226        6.98
278716       0.94
459318      14.18
310701      -0.00
202947       1.23
149777     549.89
94559        2.64
83398       34.26
457807     241.66
435443     109.53
374444      -0.00
165923      57.15
121968      23.05
334404       1.02
149245       1.59
81386       20.63
372826     130.61
59986      628.20
           ...   
272039      -0.00
100322       2.74
402584      96.45
43772      179.22
422881      -0.00
212083       4.04
47823       57.16
158646     246.71
281796      89.47
175729      39.77
244309      48.63
108285      32.36
465723     157.91
261598       0.74
193943       3.09
166965      51.08
202164       1.96
466026     885.81
158558      26.93
40483        1.88
130957      75.65
407467       0.84
297894      43.32
227582     111.08
358256    

In [83]:
gradient_boost.score(Xtest,ytest)

0.999917658524057

In [84]:
joblib.dump(gradient_boost, 'gradient_boost_model.pkl')

['gradient_boost_model.pkl']

## Messing around to get new models coming in to work regardless of values inputted

In [ ]:
pd.get_dummies()

In [61]:
try_1 = Xtest.head()

In [66]:
try_1.drop('created_date_year',axis=1,inplace=True)

/home/pipingkid/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [78]:
try_1['test2']=0

/home/pipingkid/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [116]:
try_1.shape

(5, 958)

In [115]:
Xtrain.shape

(347634, 958)

In [90]:
drop_cols(try_1,column_names)

/home/pipingkid/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [112]:
length_missing_cols = try_1.shape[1] - 958

In [114]:
for i in range(-length_missing_cols):
    try_1['{}'.format(i)]= 0

/home/pipingkid/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [109]:
drop_cols(try_1,column_names)

/home/pipingkid/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [100]:
def drop_cols(df, del_cols):  #Drop columns not in data frame and reshape 
    for col in (set(df.columns) - del_cols):
        df.drop([col], axis=1,inplace=True)

### Modeling:

In [6]:
cd ..

/home/pipingkid/Galvanize


#### Dask extension

In [2]:
import dask.dataframe as dd
import pandas as pd
import numpy as np

In [7]:
keep_cols = ['Agency','Borough','Closed Date','Community Board','Complaint Type','Created Date','Descriptor',
             'Open Data Channel Type','Status','Longitude','Latitude']                    
# # Error rasied with type being verified on these specific col, no longer needed with selecti, fixed with setting dtypes
# types={'Incident Zip': 'object',
#        'Landmark': 'object',
#        'Vehicle Type': 'object'}
%time df = dd.read_csv("311_Service_Requests_from_2010_to_Present.csv",  usecols=keep_cols)

CPU times: user 129 ms, sys: 4.42 ms, total: 133 ms
Wall time: 252 ms


In [8]:
df['Created Date'] = pd.to_datetime(df['Created Date'].compute(),utc=True)


KeyboardInterrupt: 

In [9]:
df.head()

,Created Date,Closed Date,Agency,Complaint Type,Descriptor,Status,Community Board,Borough,Open Data Channel Type,Latitude,Longitude
0,08/10/2015 09:18:31 AM,08/13/2015 12:46:56 PM,HPD,PLUMBING,STEAM PIPE/RISER,Closed,04 BRONX,BRONX,PHONE,40.840863,-73.911364
1,08/10/2015 12:20:43 PM,08/20/2015 12:40:41 PM,HPD,PLUMBING,STEAM PIPE/RISER,Closed,14 BROOKLYN,BROOKLYN,PHONE,40.636670,-73.952617
2,08/10/2015 06:16:29 PM,08/11/2015 09:15:29 AM,NYPD,Illegal Parking,Blocked Hydrant,Closed,05 QUEENS,QUEENS,ONLINE,40.732584,-73.892230
3,08/10/2015 10:50:00 PM,08/19/2015 12:45:00 AM,DEP,Noise,Noise: Construction Before/After Hours (NM1),Closed,08 BRONX,BRONX,ONLINE,40.875100,-73.910534
4,08/10/2015 10:36:00 PM,08/16/2015 12:15:00 AM,DEP,Noise,Noise: Construction Before/After Hours (NM1),Closed,02 BROOKLYN,BROOKLYN,PHONE,40.697412,-73.968683


In [16]:
df['Complaint Type'].value_counts().compute()

KeyboardInterrupt: 

In [132]:
df['new_index'] = df['Created Date']

In [135]:
# #Following Dask suggestiong to set dat to index, but keeping date col intact, however do once and make copy to persist
# df.set_index('new_idex')

,Created Date,Closed Date,Agency,Complaint Type,Descriptor,Status,Community Board,Borough,Open Data Channel Type,Latitude,Longitude
npartitions=165,,,,,,,,,,,
01/01/2010 01:00:00 AM,object,object,object,object,object,object,object,object,object,float64,float64
01/02/2013 11:27:00 AM,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...
12/30/2016 03:20:31 PM,...,...,...,...,...,...,...,...,...,...,...
12/31/2017 12:59:35 PM,...,...,...,...,...,...,...,...,...,...,...


In [121]:
import sqlite3 
engine = sqlite3.connect("/home/pipingkid/Galvanize/Energy-NYCHA/sql_tables/predictions_311.db3")

In [127]:
pd.read_sql_query('SELECT * FROM predictions_311',con=engine)

,index,agency,borough,community_board,complaint_type,created_date,descriptor,latitude,longitude,open_data_channel_type,status,unique_key,Predicted_Time_To_Close
0,1,NYPD,QUEENS,03 QUEENS,Noise - Residential,2018-09-03 00:00:23,Loud Music/Party,40.75074471873868,-73.87476796369447,MOBILE,Assigned,40176158,196.404615
1,3,DOT,MANHATTAN,01 MANHATTAN,Street Condition,2018-09-03 00:00:53,Pothole,40.707269910970616,-74.00431378701295,UNKNOWN,Open,40173118,1.733000
2,4,NYPD,BROOKLYN,17 BROOKLYN,Noise - Residential,2018-09-03 00:00:55,Loud Music/Party,40.63207531915191,-73.94078828453061,ONLINE,Open,40176183,900.146000
3,5,DSNY,BROOKLYN,12 BROOKLYN,Request Large Bulky Item Collection,2018-09-03 00:01:00,Request Large Bulky Item Collection,40.62425083314415,-73.97221140252122,PHONE,Assigned,40172583,110.543000
4,6,NYPD,QUEENS,03 QUEENS,Blocked Driveway,2018-09-03 00:01:04,Partial Access,40.75996291280347,-73.87857330674447,ONLINE,Open,40173801,326.839000
5,7,NYPD,QUEENS,08 QUEENS,Noise - Residential,2018-09-03 00:01:12,Loud Music/Party,40.710564448995946,-73.81573506370212,PHONE,Assigned,40172170,177.245615
6,8,NYPD,MANHATTAN,12 MANHATTAN,Noise - Street/Sidewalk,2018-09-03 00:01:16,Loud Music/Party,40.863693613378366,-73.92236079803905,MOBILE,Open,40172000,347.963000
7,9,NYPD,BROOKLYN,17 BROOKLYN,Noise - Residential,2018-09-03 00:01:20,Loud Music/Party,40.640767549311825,-73.95140655947002,ONLINE,Open,40176184,900.146000
8,10,NYPD,QUEENS,10 QUEENS,Blocked Driveway,2018-09-03 00:01:21,No Access,40.681973876883674,-73.82206958096853,PHONE,Assigned,40170877,293.848000
9,11,NYPD,STATEN ISLAND,01 STATEN ISLAND,Illegal Parking,2018-09-03 00:01:39,Blocked Hydrant,40.62272527581147,-74.13992689264273,ONLINE,Assigned,40175859,88.518000
